# Run this 4 times for the following data:
    1) Leptin - LHb: Vehicle vs Leptin
    2) Leptin - VTA: Vehicle vs Leptin
    3) Ghrelin - LHb: Vehicle vs Ghrelin
    4) Ghrelin - VTA: Vehicle vs Ghrelin

In [ ]:
## load required packages ##
import os
import matplotlib.pyplot as plt
import pylab as p
%matplotlib inline
import seaborn as sns
import numpy as np
import scipy.stats as stats
from os import listdir
import csv
import math
import pandas as pd
import scipy.integrate 
import scipy.io as sio
from IPython.core.display import display, HTML
from __future__ import division
import pickle
display(HTML("<style>.container { width:100% !important; }</style>"))
import pyamg
import warnings
warnings.filterwarnings('ignore')
from ipywidgets import IntProgress
from IPython.display import display
import time

In [ ]:
## EDIT THIS SECTION | FILE & DIRECTORY INFO, HOUSEKEEPING ##
basedir1 = "\\Users\\rossiadmin\\Dropbox (Stuber Lab)\\Mark\\LHA projection paper\\Data\\Fig7 hormones\\ghrelin\\LHb\\vehicle"
basedir2 = "\\Users\\rossiadmin\\Dropbox (Stuber Lab)\\Mark\\LHA projection paper\\Data\\Fig7 hormones\\ghrelin\\LHb\\ghrelin"
condition = ['vehicle','ghrelin']
projection='LHb'

lickalignfile='TRACKED_fissa_lick_align.npy'  #Lick aligned file name

fissa=['yes'] ## are the files fissa decontaminated?

filename='LHb_ghrelin' ## name to be appended to saved files/figs

## Save Figures?
save_figs=['yes']

## Fig file type
fig_style='pdf' ##png, pdf, tif

## Save data files?
save_files=['yes']

maxnumneurons = 1000 #used to initialize arrays. Should be larger than the total number of neurons
framerate=5

## Color palatte ##
cmap=sns.diverging_palette(200, 275, sep=50, as_cmap=True, center='dark') ##Green-Purple
#cmap = sns.diverging_palette(230, 5, sep=20, as_cmap=True, center='dark') ###Teal-Pink
#cmap=sns.diverging_palette(220, 35, sep=100, as_cmap=True, center='dark') ###Teal-Gold

## Colors for bar/line plots (order reflects order of 'condition' above) ##
colors = ['lightseagreen','magenta']

In [ ]:
### EDIT THIS SECTION | SET PARAMS FOR SUCROSE ANALYSIS ###
maxtrials=20    #number of trials
framespertrial=100 #frames per trial in lick align file
numpreframes=35 #frames before first lick

## normalize data by baseline?
normalize_on=['yes']
baseline=[20,35]
test=[35,65] ##used to calc significance

## END NECESSARY EDITS ##

In [ ]:
## DO NOT EDIT - set path, create output directories, save params ##
if fissa[0]=='yes':
    filename=filename+'_fissa'
else:
    filename=filename
os.chdir(basedir1)
os.chdir('..')
try:
    os.mkdir(filename+'_OUTPUT')
except OSError:
    print ("Creation of the directory %s failed" % filename+'_OUTPUT')
else:
    print ("Successfully created the directory %s " % filename+'_OUTPUT')
os.chdir(filename+'_OUTPUT')
try:
    os.mkdir('BL')
    os.mkdir('Sucrose')
    os.mkdir('Behavior')
except OSError:
    pass
%pwd

#save parameters to csv
params=['Lick align file: '+str(lickalignfile),
        'Trials: '+str(maxtrials),'Frames per trial: '+str(framespertrial),
        'Lick occurs at frame: '+str(numpreframes), 'Normalize sucrose by baseline? '+str(normalize_on[0]),
        'Sucrose baseline period (frames): '+str(baseline), 'Sucrose test period (frames): '+str(test),
        'framerate = '+str(framerate)]
with open(filename+'_analysis_parameters.csv','wb') as myfile:
    out=csv.writer(myfile,delimiter=',',dialect='excel',quoting=csv.QUOTE_ALL)
    out.writerow(params)

In [ ]:
## DO NOT EDIT -  Load files in basedir1 and basedir2 ##
def load_files(directory):
    signals_pop = np.nan*np.zeros((maxtrials,framespertrial,maxnumneurons))
    trials_pop=np.nan*np.zeros((maxnumneurons))
    data_dirs = os.walk(directory).next()[1]
    numneuronstillnow = 0
    for data_dir in data_dirs:
        try:
            signals=np.load(os.path.join(directory,data_dir,lickalignfile))
        except:
            continue
        numneurons=signals.shape[2]
        numframes=signals.shape[1]
        numtrials=signals.shape[0]
        for a in range(0,numneurons):
            signals_pop[0:numtrials,0:signals.shape[1],numneuronstillnow+a]=signals[0:maxtrials,:,a]
            trials_pop[numneuronstillnow:numneuronstillnow+a+1]=numtrials
        numneuronstillnow += numneurons
    extractedsignals=signals_pop[:,:,:numneuronstillnow]
    trials_pop=trials_pop[:numneuronstillnow]
    print '\nfiles = '+str(data_dirs)
    print 'Number of neurons = '+str(extractedsignals.shape[2])
    print 'Number of frames per trial = '+str(extractedsignals.shape[1])
    return extractedsignals,numneuronstillnow,numframes,numneurons,numtrials,trials_pop
extractedsignals1,numneuronstillnow1,numframes1,numneurons1,numtrials1,trials_pop1=load_files(basedir1)
extractedsignals2,numneuronstillnow2,numframes2,numneurons2,numtrials2,trials_pop2=load_files(basedir2)

In [ ]:
## DO NOT EDIT - z-score data if using fissa files ##
def normfissa(data):
    if filename[-5:]=='fissa':
        print '*** FISSA data detected - Normalizing ***'
        temp1=np.nan*np.zeros((data.shape))
        for a in range(data.shape[2]):
            temp1[:,:,a]=data[:,:,a]/np.nanmean(data[:,:,a])
        data=temp1
    return data
extractedsignals1=normfissa(extractedsignals1)
extractedsignals2=normfissa(extractedsignals2)

In [ ]:
## DO NOT EDIT - normalize data by baseline period ##
if normalize_on[0]=='yes': 
    def normalize_data(data):
        BLnorm=np.nan*np.zeros((data.shape))
        BLavg=np.nan*np.zeros((data.shape[0],data.shape[2]))
        for b in range(0,data.shape[2]):
            for a in range(0,data.shape[0]):
                BLavg[a,b]=np.nanmean(data[a,baseline[0]:baseline[1],b])
        for e in range(0,data.shape[2]):
            for d in range(0,data.shape[1]):
                for c in range(0,data.shape[0]):
                    BLnorm[c,d,e]=data[c,d,e]-BLavg[c,e]
        reshape_response=BLnorm
        return reshape_response
    extractedsignals1_norm=normalize_data(extractedsignals1)
    extractedsignals2_norm=normalize_data(extractedsignals2)
else:
    extractedsignals1_norm=extractedsignals1
    extractedsignals2_norm=extractedsignals2

In [ ]:
## Fig 7F,G,JK: pop average ##
ymin=-.02
ymax=.07

avg_response1=np.nanmean(extractedsignals1_norm,axis=0)
avg_response2=np.nanmean(extractedsignals2_norm,axis=0)
peak_response1=np.nanmax(avg_response1[test[0]:test[1],:],axis=0)
peak_response2=np.nanmax(avg_response2[test[0]:test[1],:],axis=0)
avg_rew_response1=np.nanmean(avg_response1[test[0]:test[1],:],axis=0)
avg_rew_response2=np.nanmean(avg_response2[test[0]:test[1],:],axis=0)
if save_files[0]=='yes':
    np.savetxt('Sucrose/'+filename+'_avg_suc_response_'+condition[0]+'.csv',avg_rew_response1.T,delimiter=',')
    np.savetxt('Sucrose/'+filename+'_avg_suc_response_'+condition[1]+'.csv',avg_rew_response2.T,delimiter=',')
    np.savetxt('Sucrose/'+filename+'_peak_suc_response_'+condition[0]+'.csv',peak_response1.T,delimiter=',')
    np.savetxt('Sucrose/'+filename+'_peak_suc_response_'+condition[1]+'.csv',peak_response2.T,delimiter=',')
    np.savetxt('Sucrose/'+filename+'_all_suc_data_'+condition[0]+'.csv',avg_response1.T,delimiter=',')
    np.savetxt('Sucrose/'+filename+'_all_suc_data_'+condition[1]+'.csv',avg_response2.T,delimiter=',')
#remove frames preceding start of baseline window
avg_response1=avg_response1[baseline[0]:,:]
avg_response2=avg_response2[baseline[0]:,:]
framespertrial=framespertrial-baseline[0]

sns.set(font_scale=1.5,style="white",rc={"lines.linewidth": 1})
sns.set_style("ticks", {"xtick.major.size": 20, "ytick.major.size": 8})
fig1,ax = plt.subplots(1,1,figsize=(8,5))
sns.tsplot(avg_response1.T,color=colors[0], condition=condition[0]+' (n = '+str(extractedsignals1.shape[2])+')', legend=True)
sns.tsplot(avg_response2.T,color=colors[1], condition=condition[1]+' (n = '+str(extractedsignals2.shape[2])+')', legend=True)
ax.set_title('Avg response '+projection)
ax.set_xlabel('Time from lick (s)')
ax.set_ylabel('dF/F')
ax.set_ylim(ymin,ymax)
ax.set_xticks(range(5, framespertrial+1, 10))
ax.set_xticklabels([str(((a-(numpreframes-baseline[0])+5)/framerate)) for a in range(0, framespertrial+1, 10)])
ax.plot([numpreframes-baseline[0], numpreframes-baseline[0]], [ymin, ymax], '--k', linewidth=1)
plt.axhline(y=0, linestyle='--', linewidth=1, color='k')
fig1.tight_layout(w_pad=5)
if save_figs[0]=='yes':
    plt.savefig(('Sucrose/'+filename+'_pop_suc_response_overlay.'+fig_style), format=str(fig_style))
    plt.show()

In [ ]:
## Fig 7E,I: heatmaps ##
def combinedheatmap(data1,data2,title,sorton):
    ymin=-.1
    ymax=.1
    if sorton=='yes':
        sort1=np.squeeze(np.argsort(np.nanmean(data1[numpreframes-baseline[0]:(numpreframes-baseline[0])+(test[1]-test[0])], axis=0, keepdims=True)))
        data1=data1[:,sort1]
        sort2=np.squeeze(np.argsort(np.nanmean(data2[numpreframes-baseline[0]:(numpreframes-baseline[0])+(test[1]-test[0])], axis=0, keepdims=True)))
        data2=data2[:,sort2]       
    data=np.concatenate((data1,data2),axis=1)
    
    ylabel1=range(0,data1.shape[1]+1,10)
    ylabel2=range(10,data2.shape[1]+1,10)
    ylabels=ylabel1+ylabel2
    
    fig2 = plt.figure(2,figsize=(8,7))
    cbar_ax = fig2.add_axes([.52, .2, .02, .6])
    cbar_ax.tick_params(width=0.5) 
    
    ax = plt.subplot(121)
    sns.set(font_scale=1.5,style="white",rc={"lines.linewidth": 1})
    sns.heatmap(data.T, cmap=cmap, vmin=ymin, vmax=ymax,linewidth=0,cbar_ax=cbar_ax,rasterized=True)
    cbar_ax.set_ylabel('dF/F')
    ax.set_title(filename+' '+str(condition[0]), fontsize=16,position=[0.5,1.05])
    ax.set_xlabel('Time from lick (s)', fontsize=14)
    ax.set_xticks(range(5, framespertrial+1, 10))
    ax.set_xticklabels([str(((a-(numpreframes-baseline[0])+5)/framerate)) for a in range(0, framespertrial+1, 10)],fontsize='16')
    ax.plot([numpreframes-baseline[0], numpreframes-baseline[0]], [0, data.shape[1]], '--w', linewidth=1)
    ax.axhline(y=data1.shape[1], color='white', linewidth=2)
    ax.set_yticks(range(0, data.shape[1]+1, 10))
    ax.set_yticklabels(ylabels,fontsize='14')
    plt.xticks(fontsize='14')
    plt.yticks(fontsize='14')
    plt.ylabel('ROIs', fontsize='16')
    
    ## control text boxes
    offset=[82,12] ##related to specified plot size. change as needed
    
    ## bottom text box
    ax.text(offset[0],offset[1], '                '+condition[0]+'                  '
            ,fontsize=16,color=colors[0],rotation=270
            ,bbox={'facecolor':colors[0],'alpha':0.3,'pad':4})
    
    ## top text box
    ax.text(offset[0],offset[1]+data1.shape[1]+1, '             '+condition[1]+'                '
            ,fontsize=16,color=colors[1],rotation=270
            ,bbox={'facecolor':colors[1],'alpha':0.5,'pad':4})
    
    fig2.tight_layout(w_pad=5)
    if save_figs[0]=='yes':
        plt.savefig(('Sucrose/'+filename+'_'+title+'.'+fig_style), format=str(fig_style))
    plt.show()
combinedheatmap(avg_response1,avg_response2,"combined_pop_heatmap_sorted",sorton='yes')

In [ ]:
## DO NOT EDIT - Function to plot bars to compare conditions ##
def bar_plot(data1,data2,ytitle,directory,param):
    if param=='Median':
        try:
            if data1.shape[1]>0:
                data1=np.nanmedian(data1,axis=1)
                data1=data1[~np.isnan(data1)]
                data2=np.nanmedian(data2,axis=1)
                data2=data2[~np.isnan(data2)]
        except:
            pass
        means=np.nan*np.zeros((1,2))
        means[:,0]=np.nanmedian(data1)
        means[:,1]=np.nanmedian(data2)
        means=np.squeeze(means)
        try:
            t,p=np.around(stats.wilcoxon(data1,data2),decimals=3)
            test='Wilcoxon Rank Sum'
        except:
            t,p=np.around(stats.ttest_ind(data1,data2),decimals=3)
            test='t-test'
        sems=np.nan*np.zeros((1,2))
        sems[:,0]= stats.sem(data1,axis=0)
        sems[:,1]=stats.sem(data2,axis=0)
        sems=np.squeeze(sems)

    else:
        test='t-test'
        try:
            if data1.shape[1]>0:
                data1=np.nanmean(data1,axis=1)
                data1=data1[~np.isnan(data1)]
                data2=np.nanmean(data2,axis=1)
                data2=data2[~np.isnan(data2)]
        except:
            pass
        means=np.nan*np.zeros((1,2))
        means[:,0]=np.nanmean(data1)
        means[:,1]=np.nanmean(data2)
        means=np.squeeze(means)
        t,p=np.around(stats.ttest_ind(data1,data2),decimals=3)
        sems=np.nan*np.zeros((1,2))
        sems[:,0]=stats.sem(data1,axis=0)
        sems[:,1]=stats.sem(data2,axis=0)
        sems=np.squeeze(sems)

    combinedata=np.nan*np.zeros((max(data1.shape[0],data2.shape[0]),2))
    combinedata[:data1.shape[0],0]=data1
    combinedata[:data2.shape[0],1]=data2
    np.savetxt(directory+'/'+filename+'_'+ytitle+'.csv', combinedata, delimiter=',')
    ind = (0,.5)
    width = 0.4
    fig,ax=plt.subplots(1,figsize=(3,6))
    bar=ax.bar(ind,means,width,yerr=sems,color=colors,error_kw={'ecolor':'black','linewidth':2})
    ax.set_ylabel(ytitle+' ('+param+')')
    ax.set_title(test+': '+str(t)+'  p = '+str(p),y=1,fontsize='16')
    ax.legend((bar[0],bar[1]),(condition[0],condition[1]),loc=[1.1,.5])
    ax.set_xticks([])
    if save_figs[0]=='yes':
        fig.savefig((directory+'/'+filename+'_'+ytitle+'.'+fig_style), format=str(fig_style),bbox_inches='tight')
    plt.show()

In [ ]:
## DO NOT EDIT - find AUC for each neuron using trapz method and plot ##
time1=15 #frames in baseline corrected data to include in AUC calculation
time2=30
def auc(data):
    AUC=np.nan*np.zeros((data.shape[1]))
    for i in range(0,data.shape[1]):
        AUC[i] = scipy.integrate.trapz(y=data[time1:time2,i])
    return AUC
auc_condition1=auc(avg_response1)
auc_condition2=auc(avg_response2)
if save_files[0]=='yes':
    np.savetxt('Sucrose/'+filename+'_AUC_sucrose_response_'+condition[0]+'.csv',auc_condition1.T,delimiter=',')
    np.savetxt('Sucrose/'+filename+'_AUC_sucrose_response_'+condition[1]+'.csv',auc_condition2.T,delimiter=',')

In [ ]:
## Fig 7H,L: AUC bar plots ##
bar_plot(auc_condition1,auc_condition2,'AUC_sucrose_response','Sucrose',param='Mean')

In [ ]:
## DO NOT EDIT - Function to compare between conditions responses (cell nums must be equal) ##
def compare2conditions(data1, data2,test_type,sig_pvalue,test_window):
    maxresponse=np.nan*np.zeros((data1.shape[0],data1.shape[2],2))
    avgresponse=np.nan*np.zeros((data1.shape[0],data1.shape[2],2))
    minresponse=np.nan*np.zeros((data1.shape[0],data1.shape[2],2))
    for a in range(data1.shape[2]):
        for b in range(data1.shape[0]):
            temp1=data1[b,test_window[0]:test_window[1],a]
            temp2=data2[b,test_window[0]:test_window[1],a]

            maxresponse[b,a,0]=np.nanmax(temp1)
            maxresponse[b,a,1]=np.nanmax(temp2)
            avgresponse[b,a,0]=np.nanmean(temp1)
            avgresponse[b,a,1]=np.nanmean(temp2)
            minresponse[b,a,0]=np.nanmin(temp1)
            minresponse[b,a,1]=np.nanmin(temp2)

    maxresponse_stats=np.nan*np.zeros((data1.shape[2],2))
    avgresponse_stats=np.nan*np.zeros((data1.shape[2],2))
    minresponse_stats=np.nan*np.zeros((data1.shape[2],2))
    for a in range(data1.shape[2]):
            if test_type=='independent':
                maxresponse_stats[a,:]=stats.ttest_ind(maxresponse[:,a,0],maxresponse[:,a,1],equal_var=False)
                avgresponse_stats[a,:]=stats.ttest_ind(avgresponse[:,a,0],avgresponse[:,a,1],equal_var=False)
                minresponse_stats[a,:]=stats.ttest_ind(minresponse[:,a,0],minresponse[:,a,1],equal_var=False)
            if test_type=='paired':
                maxresponse_stats[a,:]=stats.ttest_rel(maxresponse[:,a,0],maxresponse[:,a,1])
                avgresponse_stats[a,:]=stats.ttest_rel(avgresponse[:,a,0],avgresponse[:,a,1])
                minresponse_stats[a,:]=stats.ttest_rel(minresponse[:,a,0],minresponse[:,a,1])
            if test_type=='wilcoxon':
                maxresponse_stats[a,:]=stats.wilcoxon(maxresponse[:,a,0],maxresponse[:,a,1],zero_method='wilcox')
                avgresponse_stats[a,:]=stats.wilcoxon(avgresponse[:,a,0],avgresponse[:,a,1],zero_method='wilcox')
                minresponse_stats[a,:]=stats.wilcoxon(minresponse[:,a,0],minresponse[:,a,1],zero_method='wilcox')

    def find_sig_cells(data,stats,label):
        sig_excited_neurons_temp=np.nan*np.zeros((data.shape[1]))
        sig_inhibited_neurons_temp=np.nan*np.zeros((data.shape[1]))
        for a in range(0,data.shape[1]):
            sig_excited_neurons_temp[a] = np.logical_and(stats[a,1]<sig_pvalue,np.nanmean(data[:,a,0])<np.nanmean(data[:,a,1])) ##condition1 less than condition2 = excited
            sig_inhibited_neurons_temp[a] = np.logical_and(stats[a,1]<sig_pvalue,np.nanmean(data[:,a,0])>np.nanmean(data[:,a,1])) ##condition1 greater than condition2 = inhibited

        num_excited=np.nansum(sig_excited_neurons_temp)
        num_inhibited=np.nansum(sig_inhibited_neurons_temp)
        no_change=(data.shape[1]-(num_excited+num_inhibited))

        ##plot pie chart of responses
        labels = 'No response', 'Positive\nresponse', 'Negative\nresponse'
        colors=[(0.7, 0.7,0.7),'c',(0.84, 0.35, 0.35)]
        explode=(0, .2, .2)
        frequency_population = np.zeros((3,4)) #3 response types x 4 trial types
        frequency_response = np.array([no_change, num_excited, num_inhibited])
        frequency_population[:,0] = frequency_response
        print "Projection: "+projection
        print "Data used: "+label +" response"
        print "Test: "+test_type, "| P threshold: "+str(sig_pvalue)
        print "Excited = ",num_excited
        print "Inhibited = ",num_inhibited
        print "Null = ",no_change
        fig9,ax1 = plt.subplots(figsize=(7,5))
        ax1.set_title(filename+' '+projection+' '+label+' response between conditions', y=1)
        ax1.pie(frequency_response, explode=explode, autopct='%1.0f%%',
                shadow=True, startangle=45, colors=colors)
        plt.tight_layout()
        if save_figs[0]=='yes':
            plt.savefig(('Sucrose/'+filename+'_'+label+'_pie_between_conditions.'+fig_style), format=str(fig_style))
        plt.show()
        return sig_excited_neurons_temp, sig_inhibited_neurons_temp
    
#     excited_max,inhibited_max=find_sig_cells(maxresponse, maxresponse_stats,label='max')
#     excited_min,inhibited_min=find_sig_cells(minresponse, minresponse_stats,label='min')
    excited_avg,inhibited_avg=find_sig_cells(avgresponse, avgresponse_stats,label='avg')

    return excited_avg, inhibited_avg   

In [ ]:
## Fig S7M: Proportion responsive relative to vehicle ##
if extractedsignals1_norm.shape[2]==extractedsignals2_norm.shape[2]:
    excited_avg, inhibited_avg=compare2conditions(extractedsignals1_norm,extractedsignals2_norm,test_type='wilcoxon',sig_pvalue=0.05,test_window=test)
else:
    print "Cell numbers not equal between conditions. Cannot display graph."

# Get licking data

In [ ]:
## DO NOT EDIT - Function to align lick data to first lick after reward and plot ##
def align_licks(basedir):  
    numtrialstillnow=0
    alllickdata=np.nan*np.zeros((maxnumneurons,framespertrial))
    alllickdatarate=np.nan*np.zeros((maxnumneurons,framespertrial))
    alllickdatarew=np.nan*np.zeros((maxnumneurons,framespertrial))
    alllickdatarewrate=np.nan*np.zeros((maxnumneurons,framespertrial))
    latency=np.nan*np.zeros((maxnumneurons))
    data_dirs = os.walk(basedir).next()[1]
    licktotal=np.nan*np.zeros((len(data_dirs))) ; q=0
    for data_dir in data_dirs:
        try:
            behaviordata=sio.loadmat(os.path.join(basedir,data_dir,'behavior.mat'))
        except:
            continue
        cues=np.squeeze(behaviordata['cues'])
        licks=np.squeeze(behaviordata['licks'])
        t_fxd=np.squeeze(behaviordata['fxdpumps'])#/1000 #in seconds
        eventlog = behaviordata['eventlog']
        tempframes = eventlog[eventlog[:,0]==9,1]
        frameaveraging = 6
        framerate = 30.0/frameaveraging #for resonant scanner, maximum rate is 30Hz
        frames = tempframes[::frameaveraging]
        cuesplus=cues[cues>0]
        
        ## calculate first lick after reward. limit to licks occurring within 15s of rew delivery. 
        firstlickafterCSplus=np.nan*np.zeros((len(t_fxd)))
        for a in range(0,len(t_fxd)):
            try:
                if t_fxd[a]-licks[licks>t_fxd[a]][1] < -10000:
                    firstlickafterCSplus[a] = np.nan
                elif t_fxd[a]-licks[licks>t_fxd[a]][0] == -51:
                    firstlickafterCSplus[a] = licks[licks>t_fxd[a]][1] #use second lick to avoid solenoid artifact
                else:
                    firstlickafterCSplus[a] = licks[licks>t_fxd[a]][1] #use second lick to avoid solenoid artifact
            except:
                firstlickafterCSplus[a]=np.nan
        
        ## remove lick artifact from 'licks' ##
        removelicks=np.nan*np.zeros((0))
        for a in range(len(t_fxd)):
            templicks=licks[np.logical_and(licks>t_fxd[a], licks<firstlickafterCSplus[a])]
            removelicks=np.append(removelicks, templicks)
        licks=licks[~np.isin(licks,removelicks)]
        licktotal[q]=len(licks) ; q+=1
        
        ## calculate latency to lick after reward delivery
        latency_temp=np.nan*np.zeros((maxtrials))
        for i in range(len(t_fxd)):
            latency_temp[i]=(firstlickafterCSplus[i]-t_fxd[i])/1000
   
        ##find frame numbers
        def framenumberforevent(event):
            framenumber = np.nan*np.zeros(event.shape)
            for ie, e in enumerate(event):
                try:
                    framenumber[ie] = np.nonzero(frames<=e)[0][-1]
                except:
                    pass
            return framenumber
        framenumberforcuesplus = np.squeeze(framenumberforevent(cuesplus))
        framenumberforlicks = np.squeeze(framenumberforevent(licks))
        framenumberforCSpluslick = np.squeeze(framenumberforevent(firstlickafterCSplus))
        framenumberforrew=np.squeeze(framenumberforevent(t_fxd))
        framenumberforCSpluslick=framenumberforCSpluslick[~np.isnan(framenumberforCSpluslick)]
        lickframes=np.zeros((maxtrials*125))
        lickframesrate=np.zeros((maxtrials*125))
        framesall=np.arange(0,lickframes.shape[0])
        numtimesampleslick = 80 #How many do you want to plot around the cue?
        
        for a in range(0,len(framesall)):
            for b in range(0,len(framenumberforlicks)):
                if framenumberforlicks[b]==framesall[a]:
                    lickframes[a]=1 
                    lickframesrate[a]=lickframesrate[a]+1
        lickframesrate=lickframesrate*framerate
        alignlick_behavior = np.zeros([maxtrials,framespertrial]) #for lick probability lick aligned
        alignlick_rate = np.zeros([maxtrials,framespertrial]) #for lick rate lick aligned
        alignlick_rew=np.zeros([maxtrials,framespertrial]) #for lick probability reward aligned
        alignlick_rewrate=np.zeros([maxtrials,framespertrial]) #for lick rate reward aligned

        for i in range(framenumberforCSpluslick.shape[0]):
            try:
                alignlick_behavior[i,:]= lickframes[int(framenumberforCSpluslick[i]-baseline[0]):int(framenumberforCSpluslick[i]+numtimesampleslick-baseline[0])]
                alignlick_rate[i,:]= lickframesrate[int(framenumberforCSpluslick[i]-baseline[0]):int(framenumberforCSpluslick[i]+numtimesampleslick-baseline[0])]
                alignlick_rew[i,:]= lickframes[int(framenumberforrew[i]-baseline[0]):int(framenumberforrew[i]+numtimesampleslick-baseline[0])]
                alignlick_rewrate[i,:]= lickframesrate[int(framenumberforrew[i]-baseline[0]):int(framenumberforrew[i]+numtimesampleslick-baseline[0])]
            except:
                templen=lickframes[int(framenumberforCSpluslick[i]-baseline[0]):].shape[0]
                alignlick_behavior[i,:templen]= lickframes[int(framenumberforCSpluslick[i]-baseline[0]):]
                alignlick_rate[i,:templen]= lickframesrate[int(framenumberforCSpluslick[i]-baseline[0]):]
        numtrialstillnow += maxtrials
    
        alllickdata[numtrialstillnow-maxtrials:numtrialstillnow,:]=alignlick_behavior[:,:]
        alllickdatarate[numtrialstillnow-maxtrials:numtrialstillnow,:]=alignlick_rate[:,:]
        alllickdatarew[numtrialstillnow-maxtrials:numtrialstillnow,:]=alignlick_rew[:,:]
        alllickdatarewrate[numtrialstillnow-maxtrials:numtrialstillnow,:]=alignlick_rewrate[:,:]
        latency[numtrialstillnow-maxtrials:numtrialstillnow]=latency_temp
    
    latency=latency[:numtrialstillnow];latency=latency[~np.isnan(latency)]
    alllickdata=alllickdata[:numtrialstillnow,:]
    alllickdatarew=alllickdatarew[:numtrialstillnow,:]
    alllickdatarate=alllickdatarate[:numtrialstillnow,:]
    alllickdatarewrate=alllickdatarewrate[:numtrialstillnow,:]
    return alllickdata,alllickdatarew,latency,licktotal,alllickdatarate,alllickdatarewrate  
        
alignlicks1,rewalignlicks1,latency1,licktotal1,alignlickrate1,rewalignrate1=align_licks(basedir1)
alignlicks2,rewalignlicks2,latency2,licktotal2,alignlickrate2,rewalignrate2=align_licks(basedir2)

In [ ]:
## Fig 7C-D: Latency to lick ##
fig17=bar_plot(latency1,latency2,'latency_to_lick','Behavior',param='Mean')

In [ ]:
## DO NOT EDIT - Funciton to plot lick data ##
def plotlickdata(data1,data2,xtitle,ylim,ylabel):
    fontsize=20
    sns.set(font_scale=1.5,style="white",rc={"lines.linewidth": 1})
    sns.set_style("ticks", {"xtick.major.size": 0, "ytick.major.size": 8})
    f,ax = plt.subplots(1,1,figsize=(10,5))
    sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)
    sns.tsplot(data1[:,:], color=colors[0], condition=condition[0], legend=True)
    sns.tsplot(data2[:,:], color=colors[1], condition=condition[1], legend=True)
    ax.set_title(filename+' lick rate '+projection,fontsize=fontsize)
    ax.set_xlabel(xtitle+' (s)',fontsize=fontsize)
    ax.set_ylabel(ylabel,fontsize=fontsize)
    ax.set_yticks(np.arange(ylim[0],ylim[1]+1,ylim[1]/2))
    ax.set_yticklabels(np.arange(ylim[0],ylim[1]+1,ylim[1]/2),fontsize=fontsize)
    ax.set_xticks(range(10, framespertrial+1, 10))
    ax.set_xticklabels([str(((a-(numpreframes-baseline[0])+5)/framerate)) for a in range(0, framespertrial+1, 10)],fontsize=fontsize)
    ax.set_ylim([ylim[0],ylim[1]+.05])
    f.tight_layout(w_pad=5)
    if save_figs[0]=='yes':
        plt.savefig(('Behavior/'+filename+'_'+xtitle+'_'+ylabel+'.'+fig_style), format=str(fig_style))
        plt.show()

In [ ]:
## Fig S7K-L
plotlickdata(alignlickrate1,alignlickrate2,'Time from first lick',ylim=[0,8],ylabel='Licks per s')